In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\diplo\AppData\Local\Temp\ipykernel_17020\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [3]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"C:\Users\diplo\Desktop\Lab\lab-natural-language-processing\data\kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [14]:
# Your code
from sklearn.model_selection import train_test_split
data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)

## Data Preprocessing

In [6]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [15]:
# Your code
def clean_html(text):
    # Remove inline JavaScript and CSS
    import re
    text = re.sub(r'<(script|style).*?>.*?(</\1>)', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    return text
# Apply to both training and validation data
data_train['clean_text'] = data_train['text'].apply(clean_html)
data_val['clean_text'] = data_val['text'].apply(clean_html)


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [20]:
# Your code
import re
def clean_text(text):
    # Remove special characters and punctuation
    text = re.sub(r'[^\w\s]', ' ', text)
    
    # Remove digits
    text = re.sub(r'\d+', '', text)
    
    # Remove single characters (standalone)
    text = re.sub(r'\b\w\b', '', text)
    
    # Remove single characters from the start of words
    text = re.sub(r'\s\w\s+', ' ', text)
    
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    
    # Remove the 'b' prefix that sometimes appears in byte strings
    text = re.sub(r'\bb\b', '', text)
    
    # Convert to lowercase
    text = text.lower()
    
    return text

# Combine both steps
data_train['preprocessed_text'] = data_train['clean_text'].apply(clean_text)
data_val['preprocessed_text'] = data_val['clean_text'].apply(clean_text)


## Now let's work on removing stopwords
Remove the stopwords.

In [21]:
# Your code
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    words = text.split()
    filtered = [word for word in words if word not in stop_words]
    return " ".join(filtered)

data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(remove_stopwords)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(remove_stopwords)


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [22]:
# Your code
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import nltk

nltk.download('wordnet')       
nltk.download('omw-1.4')       

lemmatizer = WordNetLemmatizer()
def lemmatize_text(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

data_train['preprocessed_text'] = data_train['preprocessed_text'].apply(lemmatize_text)
data_val['preprocessed_text'] = data_val['preprocessed_text'].apply(lemmatize_text)



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diplo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\diplo\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [23]:
# Your code
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X_counts = vectorizer.fit_transform(data_train['text'])

word_counts = pd.DataFrame(X_counts.toarray(), columns=vectorizer.get_feature_names_out())
word_counts['label'] = data_train['label'] 

ham_words = word_counts[word_counts['label'] == 'ham'].drop(columns='label').sum().sort_values(ascending=False).head(10)
spam_words = word_counts[word_counts['label'] == 'spam'].drop(columns='label').sum().sort_values(ascending=False).head(10)

print("Top 10 words in ham emails:")
print(ham_words)
print("\nTop 10 words in spam emails:")
print(spam_words)


Top 10 words in ham emails:
00                                       0
oypemf5fxpnyvpi1pwkaxr99bqo1v7jefyv7j    0
oxxup8afmkwpobmkix5g9akwhvnvxw1hg        0
oxhxq4fgxjcd                             0
ox6                                      0
owp                                      0
ownsince                                 0
owns                                     0
ownmerits                                0
owninterests                             0
dtype: int64

Top 10 words in spam emails:
00                                       0
oypemf5fxpnyvpi1pwkaxr99bqo1v7jefyv7j    0
oxxup8afmkwpobmkix5g9akwhvnvxw1hg        0
oxhxq4fgxjcd                             0
ox6                                      0
owp                                      0
ownsince                                 0
owns                                     0
ownmerits                                0
owninterests                             0
dtype: int64


## Extra features

In [24]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

data_train['money_mark'] = data_train['preprocessed_text'].str.contains(money_simbol_list)*1
data_train['suspicious_words'] = data_train['preprocessed_text'].str.contains(suspicious_words)*1
data_train['text_len'] = data_train['preprocessed_text'].apply(lambda x: len(x)) 

data_val['money_mark'] = data_val['preprocessed_text'].str.contains(money_simbol_list)*1
data_val['suspicious_words'] = data_val['preprocessed_text'].str.contains(suspicious_words)*1
data_val['text_len'] = data_val['preprocessed_text'].apply(lambda x: len(x)) 

data_train.head()

,text,label,clean_text,preprocessed_text,money_mark,suspicious_words,text_len
29,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",1,"----------- REGARDS, MR NELSON SMITH.KINDLY RE...",regard mr nelson smith kindly reply private em...,1,0,79
535,I have not been able to reach oscar this am. W...,0,I have not been able to reach oscar this am. W...,able reach oscar supposed send pdb receive,1,0,42
695,; Huma Abedin B6I'm checking with Pat on the 5...,0,; Huma Abedin B6I'm checking with Pat on the 5...,huma abedin bi checking pat work jack jake res...,1,0,79
557,I can have it announced here on Monday - can't...,0,I can have it announced here on Monday - can't...,announced monday today,1,0,22
836,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,1,BANK OF AFRICAAGENCE SAN PEDRO14 BP 1210 S...,bank africaagence san pedro bp san pedro cote ...,1,1,1103


## How would work the Bag of Words with Count Vectorizer concept?

In [25]:
# Your code
texts = [
    "My name is Simbiat",
    "I am an Ironhacker"
]

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

print(vectorizer.get_feature_names_out())
print(X.toarray())


['am' 'an' 'ironhacker' 'is' 'my' 'name' 'simbiat']
[[0 0 0 1 1 1 1]
 [1 1 1 0 0 0 0]]


## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [27]:
# Your code
from sklearn.feature_extraction.text import TfidfVectorizer

# 1. Load the vectorizer
tfidf_vectorizer = TfidfVectorizer()

# 2. Vectorize the dataset
X_tfidf = tfidf_vectorizer.fit_transform(data['text'])

# 3. Print the shape of the vectorized dataset
print("TF-IDF matrix shape:", X_tfidf.shape)


TF-IDF matrix shape: (1000, 25592)


## And the Train a Classifier?

In [28]:
# Your code
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix

# 1. Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X_tfidf, data['label'], test_size=0.2, random_state=42
)

# 2. Initialize the MultinomialNB classifier
nb_classifier = MultinomialNB()

# 3. Train the model
nb_classifier.fit(X_train, y_train)

# 4. Make predictions
y_pred = nb_classifier.predict(X_test)

# 5. Evaluate the model
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


[[102  23]
 [  0  75]]
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       125
           1       0.77      1.00      0.87        75

    accuracy                           0.89       200
   macro avg       0.88      0.91      0.88       200
weighted avg       0.91      0.89      0.89       200



### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# Your code